# Deploy LHTM-Opt Model Package from AWS Marketplace 




**LHTM-Opt** is an instruction-tuned Japanese Language Large Model developed by alt Inc., which has excellent Japanese knowledge and can be applied in various natural language processing (NLP) tasks. alt Inc. is a venture firm with the mission of freeing humankind from non-creative/unproductive labor through the creation of P.A.I.® ️ (Personal Artificial Intelligence) and AI clones.

Lightweight and Deployable: With 7B model size, our LLM is designed to be lightweight, ensuring ease of deployment.

Benchmark Excellence: LHTM-Opt obtained competitive scores on the JGLUE and Rakuda benchmarks, which are benchmarks for Japanese LLMs. These scores are a testament to our model's understanding, reasoning, and generation capabilities.

Ideal for RAG Applications: LHTM-Opt can enhance question answering systems, content creation tools, and more by providing contextually relevant, accurate, and coherent responses.

Seamless Integration: Published on AWS Marketplace, our Japanese LLM is ready for immediate deployment.


This sample notebook shows you how to deploy [LHTM-Opt](https://aws.amazon.com/marketplace/pp/prodview-nw62wpreit442) using Amazon SageMaker.


## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [LHTM-Opt](https://aws.amazon.com/marketplace/pp/prodview-nw62wpreit442). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Delete the endpoint](#D.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [LHTM-Opt](https://aws.amazon.com/marketplace/pp/prodview-nw62wpreit442).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn = (
    "<Customer to specify Model package ARN corresponding to their AWS region>"
)

In [ ]:
import json

import boto3
import sagemaker as sage
from sagemaker import ModelPackage, get_execution_role

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "lhtmopt"

content_type = "application/json"

real_time_inference_instance_type = "ml.p3.2xlarge"
batch_transform_inference_instance_type = "ml.p3.2xlarge"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
# Simple question answering example
payload = {
    "prompt": "質問： 博多の食べ物は何が有名ですか？\n答え：",
}

In [ ]:
# Simple question answering example with all params
payload = {
    "prompt": "質問： 日本で一番高い山は何ですか？簡潔に教えてください。\n答え：",
    "max_new_tokens": 50,
    "temperature": 0.2,
    "top_k": 20,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.2,
    "skip_prompt": True,
}

In [ ]:
# Llama2 chat template example
payload = {
    "prompt": """[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

日本はどこが一番美人女性が多いと言われていますか？ [/INST]""",
    "temperature": 0.3,
    "top_k": 99,
}

### C. Perform real-time inference

In [ ]:
# real-time inference
def invoke_endpoint(endpoint_name, payload):
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        Body=json.dumps(payload),
    )

    return json.loads(response["Body"].read().decode("utf-8"))

In [ ]:
result = invoke_endpoint(model_name, payload)

In [ ]:
print(f"Result: {result}")

### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(
    transform_input_folder, key_prefix=model_name
)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

